In [1]:
import os
import sys
import glob
import numpy

import multiprocessing 
import concurrent.futures

In [2]:
def grep_recur(base_path, pattern="*.*"):
    from itertools import chain
    sub_greps = list(chain(*[grep_recur(dp, pattern) for dp in grep_dirs(base_path)]))
    return grep_files(base_path, pattern) + sub_greps

def grep_files(base_path, pattern="*.*"):
    import glob
    return glob.glob("{}/{}".format(base_path, pattern))

def grep_dirs(base_path):
    file_paths = [os.path.join(base_path, name) for name in os.listdir(base_path)]
    return [p for p in file_paths if os.path.isdir(p)]

In [3]:
base_dir_path = "N:\everyone"
out_dir_path = "N:\ps"

In [4]:
def write_to_npy(pid, frame_paths, out_dir_path):
    frame_indexes, frames = [], []
    for path in frame_paths:
        with open(path, 'rb') as f:
            frame_idx = path[-9:-4]
            frame = f.read()
            frames.append(frame)
            frame_indexes.append(frame_idx)
            
    out_path = os.path.join(out_dir_path, "frame-{}.npz".format(pid))
    numpy.savez_compressed(out_path, frames=frames, index=frame_indexes)
    # np.save(out_path, frames)

In [7]:
def write_frame_to_npy(paths, out_dir_path):
    for profile_path in paths:
        pid = profile_path[-5:]
        profile_frame_root_path = os.path.join(profile_path, 'frames')
        frame_paths = grep_files(profile_frame_root_path, "*.jpg")
        
        write_to_npy(pid, frame_paths, out_dir_path)
        print("To npy profile[{}]".format(pid))

In [8]:
profile_paths = grep_dirs(base_dir_path)

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as e:
    for i in range(0, len(profile_paths), 25):
        paths = profile_paths[i:i+25]
        e.submit(write_frame_to_npy, paths, out_dir_path)

To npy profile[00002]
To npy profile[00339]
To npy profile[00003]
To npy profile[00005]
To npy profile[00006]
To npy profile[00475]
To npy profile[00658]
To npy profile[00791]
To npy profile[00342]
To npy profile[00165]
To npy profile[00226]
To npy profile[00010]
To npy profile[00024]
To npy profile[00661]
To npy profile[00173]
To npy profile[00343]
To npy profile[00227]
To npy profile[00126]
To npy profile[00028]
To npy profile[00274]
To npy profile[00795]
To npy profile[00833]
To npy profile[00743]
To npy profile[00881]
To npy profile[00565]
To npy profile[00696]
To npy profile[00477]
To npy profile[00663]
To npy profile[00699]
To npy profile[00519]
To npy profile[00033]
To npy profile[00745]
To npy profile[00616]
To npy profile[00178]
To npy profile[00617]
To npy profile[00666]
To npy profile[00127]
To npy profile[00034]
To npy profile[00087]
To npy profile[00089]
To npy profile[00097]
To npy profile[00228]
To npy profile[00229]
To npy profile[00098]
To npy profile[00882]
To npy pro

In [ ]:
frames = np.load(os.path.join(out_dir_path, 'frame-00002.npy'))